In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import os
import glob
from pprint import pprint
from itertools import islice
import random

# Set the max_columns option to None to display all columns
pd.set_option('display.max_columns', None)

In [2]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 


In [3]:
economic_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/df_foranalysis.csv')
us_counties = gp.read_file("/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/tl_2019_us_county/tl_2019_us_county.shp")  # CA counties


In [4]:
us_counties['GEOID'] = us_counties['GEOID'].astype(int)

In [8]:
# merge
merged_df = economic_df.merge(us_counties[['GEOID', 'geometry']], left_on='geofips', right_on='GEOID', how='left')

merged_df = merged_df.drop(columns='GEOID')

gdf_merged = gp.GeoDataFrame(merged_df, geometry='geometry')

In [9]:
gdf_merged.head()

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode,avg_growthrate_5y,avg_growthrate_10y,avg_growthrate_20y,geometry
0,4,8,4,1,Arizona,Apache County,4001,NaN,0.027246,0.020066,0.030087,0.030314,0.020613,0.068930,-0.005125,-0.011229,-0.027159,-0.011921,-0.012065,-0.022241,0.003471,0.012141,-0.009875,0.001843,0.000972,-0.011801,0.059831,0.006599,ann_growthrate,40,0.011489,0.002888,0.008535,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
1,4,8,4,1,Arizona,Apache County,4001,34632.0,35602.000000,36331.000000,37458.000000,38629.000000,39442.000000,42362.000000,42146.000000,41678.000000,40576.000000,40098.000000,39620.000000,38758.000000,38893.000000,39371.000000,38986.000000,39058.000000,39096.000000,38640.000000,41099.000000,41372.000000,jobs_per_100k,30,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
2,4,8,4,1,Arizona,Apache County,4001,67863.0,67319.000000,68072.000000,68161.000000,68521.000000,69390.000000,69602.000000,69883.000000,71008.000000,71828.000000,72181.000000,72231.000000,72333.000000,71808.000000,71026.000000,71418.000000,71538.000000,71819.000000,71920.000000,65911.000000,65385.000000,population,20,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
3,4,8,4,1,Arizona,Apache County,4001,23502.0,23967.000000,24731.000000,25532.000000,26469.000000,27369.000000,29485.000000,29453.000000,29595.000000,29145.000000,28943.000000,28618.000000,28035.000000,27928.000000,27964.000000,27843.000000,27941.000000,28078.000000,27790.000000,27089.000000,27051.000000,total employment,10,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1..."
4,4,8,4,3,Arizona,Cochise County,4003,NaN,-0.011890,0.027794,0.000598,0.017390,0.005264,0.020469,-0.020966,-0.019845,-0.016014,-0.017262,-0.014872,-0.013858,0.001048,-0.002811,-0.005579,0.004453,-0.007618,0.000961,0.032896,-0.007004,ann_growthrate,40,0.004737,-0.001238,-0.001342,"POLYGON ((-110.45230 31.74455, -110.45230 31.7..."


In [17]:
gdf.head(2)

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode,avg_growthrate_5y,avg_growthrate_10y,avg_growthrate_20y,geometry,FIPS,RUCC_2013,Description,rucc_broad
0,4,8,4,1,Arizona,Apache County,4001,NaN,0.027246,0.020066,0.030087,0.030314,0.020613,0.06893,-0.005125,-0.011229,-0.027159,-0.011921,-0.012065,-0.022241,0.003471,0.012141,-0.009875,0.001843,0.000972,-0.011801,0.059831,0.006599,ann_growthrate,40,0.011489,0.002888,0.008535,"POLYGON ((-109.84327 35.17040, -109.84370 35.1...",4001,6.0,"Nonmetro - Urban population of 2,500 to 19,999...",nonmetro
1,4,8,4,1,Arizona,Apache County,4001,34632.0,35602.000000,36331.000000,37458.000000,38629.000000,39442.000000,42362.00000,42146.000000,41678.000000,40576.000000,40098.000000,39620.000000,38758.000000,38893.000000,39371.000000,38986.000000,39058.000000,39096.000000,38640.000000,41099.000000,41372.000000,jobs_per_100k,30,NaN,NaN,NaN,"POLYGON ((-109.84327 35.17040, -109.84370 35.1...",4001,6.0,"Nonmetro - Urban population of 2,500 to 19,999...",nonmetro


In [18]:
gdf = gdf.rename(columns={'avg_growthrate_5y': 'avg_gr_5y'})
gdf = gdf.rename(columns={'avg_growthrate_10y': 'avg_gr_10y'})
gdf = gdf.rename(columns={'avg_growthrate_20y': 'avg_gr_20y'})

In [19]:
# subset to just the average growth rate data for ease of joining tables in ArcGIS
gdf_avggrowthrate = gdf[gdf['linecode'] == 40].copy()
gdf_avggrowthrate.to_file('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/geodf_growthrate.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# gdf.to_file('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/geodf_foranalysis.shp', driver='ESRI Shapefile')